In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import re
import sys
from pathlib import Path
from natsort import natsorted
from sklearn.metrics import roc_curve
from matplotlib.ticker import LogLocator

sys.path.insert(0, str(Path().resolve().parents[1]))
from src.config import *

In [ ]:
# Choose config file
config = get_config("config.yaml")

# Get results path
res_path = get_base_path(config) / config["results_path"]

# Choose where to save plots
plots_path = get_base_path(config) / "plots"
create_clean_dir(plots_path)

In [ ]:
# Split data set accordingly to a probability threshold
def split_data(data: pd.DataFrame, col: str, threshold: float = 0.5) -> tuple:

    # Split results based on probabilities
    cert_idx = data[(data[col] > threshold)].index
    data_cert = data.iloc[cert_idx]
    data_uncert = data[~data.index.isin(cert_idx)]

    return data_cert, data_uncert


# Accuracy function for BN
def get_acc_bn(data: pd.DataFrame, col: str, vs_col: str) -> float:

    return sum(data[col] == data[vs_col]) / len(data)

In [ ]:
res_path = get_base_path(config) / config["results_path"]
dirs = natsorted(
    [f"{res_path}/{dir}" for dir in os.listdir(f"{res_path}/") if "results_" in dir]
)

res = {
    "ess": [],
    "eps": [],
    "acc_cn_cert": [],
    "acc_cn_uncert": [],
    "acc_cn_tot": [],
    "acc_noisy_bn": [],
    "cert_cn": [],
}

roc = {
    "roc_cn_cert": dict(),
    "roc_cn_uncert": dict(),
    "roc_cn_tot": dict(),
    "roc_noisy_bn": dict(),
}

# For each ess...
for dir in dirs:

    # Get results
    files = [f for f in os.listdir(dir) if ".csv" in f]
    data = pd.concat([pd.read_csv(dir + "/" + f) for f in files])
    data.reset_index(inplace=True)
    data["bn_noisy_probs_1"] = data.apply(
        lambda row: (
            row["bn_noisy_probs"]
            if row["bn_noisy_mpes"] == 1
            else (1 - row["bn_noisy_probs"])
        ),
        axis=1,
    )
    data["cn_probs_1"] = data.apply(
        lambda row: row["cn_probs"] if row["cn_mpes"] == 1 else row["cn_probs_alt"],
        axis=1,
    )

    # Store ess
    reg = re.search("nodes(\d+)_ess(\d+)", dir)
    n_nodes = reg.group(1)
    ess = reg.group(2)

    # Store avg of eps and std
    with open(dir + "/exp_meta.txt", "r") as f:
        eps_vec = [
            float(re.search("Eps: (.+)\n", line).group(1))
            for line in f
            if "Eps: " in line
        ]
    eps = (float(np.mean(eps_vec)), float(np.std(eps_vec)))

    # Split CN results based on probabilities
    data_cert, data_uncert = split_data(data, "cn_probs", 0.5)

    # Compute accuracies
    vs = "gt"

    acc_cn_cert = (
        get_acc_bn(data_cert, f"{vs}_mpes", "cn_mpes") if len(data_cert) > 0 else None
    )
    acc_cn_uncert = (
        get_acc_bn(data_uncert, f"{vs}_mpes", "cn_mpes")
        if len(data_uncert) > 0
        else None
    )
    acc_cn_tot = get_acc_bn(data, f"{vs}_mpes", "cn_mpes")
    acc_noisy_bn = get_acc_bn(data, f"{vs}_mpes", "bn_noisy_mpes")

    # Compute CN certainty
    cert_cn = sum(data["cn_probs"] > 0.5) / len(data)

    # Compute ROC
    roc_cn_cert = roc_curve(data_cert[f"{vs}_mpes"], data_cert["cn_probs_1"])
    roc_cn_uncert = roc_curve(data_uncert[f"{vs}_mpes"], data_uncert["cn_probs_1"])
    roc_cn_tot = roc_curve(data[f"{vs}_mpes"], data["cn_probs_1"])
    roc_noisy_bn = roc_curve(data[f"{vs}_mpes"], data["bn_noisy_probs_1"])

    # Store results
    for key in res.keys():
        res[key].append(eval(key))
    for key in roc.keys():
        roc[key][ess] = eval(key)

    # Debug
    assert (data["cn_probs"] >= data["cn_probs_alt"]).all()
    assert (data["bn_noisy_probs"] >= 0.5).all()
    assert (data["bn_probs"] >= 0.5).all()
    assert len(data) == len(pd.read_csv(dir + "/" + files[0])) * len(files)

# Debug
length = len(res["ess"])
for key in res.keys():
    assert len(res[key]) == length
for key in roc.keys():
    assert len(roc[key]) == length
assert res["ess"] == sorted(res["ess"])

In [ ]:
# Style
plt.style.use("seaborn-v0_8-paper")
plt.rcParams.update(
    {
        "font.size": 9,
        "axes.labelsize": 9,
        "axes.titlesize": 9,
        "legend.fontsize": 7,
        "xtick.labelsize": 8,
        "ytick.labelsize": 8,
        "lines.linewidth": 0.8,
        "figure.dpi": 300,
        "savefig.dpi": 300,
        "axes.edgecolor": "black",
        "axes.linewidth": 0.8,
        "text.usetex": True,
    }
)

In [ ]:
# Ess vs eps
fig, ax = plt.subplots(1, 1, figsize=(5, 3))

eps_mean = np.array([x[0] for x in res["eps"]])
ax.semilogy(res["ess"], eps_mean, "-o", label="Mean", markersize=4)
ax.set_xlabel("S")
ax.set_ylabel("$\epsilon$")
ax.set_title("S vs $\epsilon$")
ax.set_ylim([1e-5, 10])
ax.set_yticks([4, 1, 1e-1, 1e-2, 1e-3, 1e-4])
ax.set_yticklabels(["4", "1", "1e-1", "1e-2", "1e-3", "1e-4"])
ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs="auto"))
ax.tick_params(axis="y", which="minor", length=2, width=0.5)
ax.tick_params(axis="y", which="major", length=3, width=0.9)
ax.grid(True, which="major", linestyle="-", linewidth=0.5, color="#bfbfbf", zorder=1)
# ax.grid(True, which='minor', linestyle='-', linewidth=0.5, color='#bfbfbf', zorder=1)
ax.legend(loc="best")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()
fig.savefig(
    f"{plots_path}/s_vs_eps.pdf", dpi=1200, bbox_inches="tight", transparent=False
)

In [ ]:
# Ess vs CN certainty
fig, ax = plt.subplots(1, 1, figsize=(5, 3))

ax.plot(res["ess"], res["cert_cn"], "-o", markersize=4)
ax.set_xlabel("S")
ax.set_ylabel("Ratio of (maxmin CN prob. $> 0.5$)")
ax.set_title("CN certainty")
ax.grid(True, which="major", linestyle="-", linewidth=0.5, color="#bfbfbf", zorder=1)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()
fig.savefig(
    f"{plots_path}/cn_certainty.pdf", dpi=1200, bbox_inches="tight", transparent=False
)

In [ ]:
# Accuracy
fig, ax = plt.subplots(1, 1, figsize=(5, 3))

labels = {
    "acc_noisy_bn": "Noisy BN",
    "acc_cn_tot": "CN (total)",
    "acc_cn_cert": "CN (certain)",
    "acc_cn_uncert": "CN (uncertain)",
}
for key in res.keys():
    if "acc" in key:
        ax.plot(res["ess"], res[key], "-o", label=labels[key], markersize=4)

ax.set_xlabel("S")
ax.set_ylabel("Accuracy")
ax.set_title("Accuracy")
ax.legend(loc="best")
ax.grid(True, which="major", linestyle="-", linewidth=0.5, color="#bfbfbf", zorder=1)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()
fig.savefig(
    f"{plots_path}/accuracy.pdf", dpi=1200, bbox_inches="tight", transparent=False
)

In [ ]:
roc.keys()

In [ ]:
# ROC curves
fig, axes = plt.subplots(2, 3, figsize=(16, 8))

labels = {
    "roc_cn_cert": "CN (certain)",
    "roc_cn_uncert": "CN (uncertain)",
    "roc_cn_tot": "CN (total)",
    "roc_noisy_bn": "Noisy BN",
}

i = 0
for ess in res["ess"]:
    ax = axes.flatten()[i]

    for key in roc.keys():
        fpr, tpr, _ = roc[key][ess]
        ax.plot(fpr, tpr, label=labels[key], linewidth=1.3)

    ax.plot(
        [0, 1],
        [0, 1],
        color="gray",
        linestyle="dashed",
        linewidth=1.3,
        label="baseline",
    )
    ax.set_xlabel("FPR")
    ax.set_ylabel("TPR")
    ax.set_title(f"ROC (S = {ess})")
    ax.grid(
        True, which="major", linestyle="-", linewidth=0.5, color="#bfbfbf", zorder=1
    )

    i += 1

plt.legend(loc="best")
plt.subplots_adjust(hspace=0.5)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()
fig.savefig(f"{plots_path}/roc.pdf", dpi=1200, bbox_inches="tight", transparent=False)